In [10]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

tf.compat.v1.enable_eager_execution()  # For TF 1.x users


In [ ]:
train_dir = "C:/Users/angir/Downloads/train_data/train_data"  # e.g., './dataset/train_data'
test_dir = "C:/Users/angir/Downloads/test_data/test_data"  # e.g., './dataset/test_data'

In [ ]:



train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)


Found 124 images belonging to 16 classes.
Found 26 images belonging to 16 classes.
Found 91 images belonging to 16 classes.


In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='softmax'))  # Adjust to your number of classes

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=125
)



Epoch 1/125
3/3 ━━━━━━━━━━━━━━━━━━━━ 14s 5s/step - accuracy: 0.0828 - loss: 2.7797 - val_accuracy: 0.1154 - val_loss: 2.6982
Epoch 2/125
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.0312 - loss: 2.8291

c:\Users\angir\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.0312 - loss: 2.8291 - val_accuracy: 0.1538 - val_loss: 2.7033
Epoch 3/125
3/3 ━━━━━━━━━━━━━━━━━━━━ 28s 12s/step - accuracy: 0.1449 - loss: 2.7131 - val_accuracy: 0.1538 - val_loss: 2.6632
Epoch 4/125
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - accuracy: 0.1250 - loss: 2.6908 - val_accuracy: 0.1538 - val_loss: 2.6660
Epoch 5/125
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.1289 - loss: 2.6027 - val_accuracy: 0.1538 - val_loss: 2.6234
Epoch 6/125
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.1786 - loss: 2.5720 - val_accuracy: 0.1538 - val_loss: 2.6473
Epoch 7/125
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.1877 - loss: 2.5932 - val_accuracy: 0.2308 - val_loss: 2.6568
Epoch 8/125
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.1875 - loss: 2.5167 - val_accuracy: 0.2692 - val_loss: 2.7473
Epoch 9/125
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.1953 - loss: 2.3845 - val_accuracy: 0.2308 - val_loss: 2.9139
Epoch 10/125
3/

In [15]:
# === 7. Save the trained model ===
model_save_path = 'saved_models/bird_classifier_model.h5'  # Or any path/folder you prefer

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

# Save the model
model.save(model_save_path)
print(f"Model saved at: {model_save_path}")


Model saved at: saved_models/bird_classifier_model.h5


In [ ]:
n_classes = train_generator.num_classes

model_lstm = Sequential([
    Reshape((150, 150 * 3), input_shape=(150, 150, 3)),
    LSTM(256, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(n_classes, activation='softmax')
])

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history_lstm = model_lstm.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)

print("\n✅ LSTM Model Evaluation:")
model_lstm.evaluate(test_generator)

In [ ]:
import numpy as np

# Predict with CNN
y_pred_cnn = model_cnn.predict(test_generator)
y_pred_cnn_classes = np.argmax(y_pred_cnn, axis=1)

# Predict with LSTM
y_pred_lstm = model_lstm.predict(test_generator)
y_pred_lstm_classes = np.argmax(y_pred_lstm, axis=1)

# True labels
y_true = test_generator.classes

# Print first 10 predictions vs actual
print("\nFirst 10 CNN Predictions vs Actual:")
print(list(zip(y_pred_cnn_classes[:10], y_true[:10])))

print("\nFirst 10 LSTM Predictions vs Actual:")
print(list(zip(y_pred_lstm_classes[:10], y_true[:10])))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os

# === 1. Load the saved model ===+
model = load_model('saved_models/bird_classifier_model.h5')

# === 2. Load and preprocess your test image ===
img_path = 'C:/Users/angir/Downloads/test_data/test_data/blasti/DSC_6411.jpg'  # <--- your image path
img = image.load_img(img_path, target_size=(150, 150))  # use (224, 224) if trained with larger images
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # normalize
img_array = np.expand_dims(img_array, axis=0)  # make shape (1, 150, 150, 3)

# === 3. Predict ===
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]

# === 4. Load class labels from training data directory ===
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_path = 'C:/Users/angir/Downloads/train_data/train_data'  # <-- your training folder

# Create a temp generator just to get class indices
datagen = ImageDataGenerator(rescale=1./255)
temp_generator = datagen.flow_from_directory(
    train_data_path,
    target_size=(150, 150),
    batch_size=1,
    class_mode='categorical'
)

class_indices = temp_generator.class_indices
labels = dict((v, k) for k, v in class_indices.items())

# === 5. Get predicted class label ===
predicted_label = labels[predicted_class_index]

print(f"\n✅ Predicted Bird Species: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Found 150 images belonging to 16 classes.

✅ Predicted Bird Species: blasti
